In [1]:
%config IPCompleter.greedy=True


In [2]:
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [3]:
url = 'https://www.zapimoveis.com.br/venda/casas/sp+guaruja/'



user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59'
                  

headers={'User-Agent':user_agent,} 
request=urllib.request.Request(url,None,headers)

response = urlopen(request)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')

In [4]:
soup = BeautifulSoup(html, 'html.parser')


# Localizando 

In [5]:
anuncio = soup.find('div', {'class':'card-container'})
anuncio.find('p', class_='simple-card__price').get_text().replace('\n','').replace(' ','')

'R$134.990'

# Scraping da pagina inteira

In [6]:
cards = []
card = {}

In [7]:
anuncios = soup.find('div', class_='listings__container').findAll('div', class_='card-container')

for anuncio in anuncios:
    card = {}
    
    #acha o valor do imóvel
    card['value'] = anuncio.find('p', class_='simple-card__price').get_text().replace('\n','').replace(' ','')
    
    #acha as características do imóvel
    infos = anuncio.find('ul', class_='feature__container simple-card__amenities').findAll('li')
    for info in infos:
        card[info.get('class')[2].split('-')[-1]] = info.get_text().split()[0]
        
    
    
    cards.append(card)

## Agora que temos os anuncios da página 1 inteira vamos para o próximo passo

# Scraping do Site inteiro

In [8]:
cards = []
card = {}

In [9]:
for i in range(70):
    url = 'https://www.zapimoveis.com.br/venda/casas/sp+guaruja/?pagina='+ str(i+1)



    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59'


    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(url,None,headers)

    response = urlopen(request)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    anuncios = soup.find('div', class_='listings__container').findAll('div', class_='card-container')
    
    for anuncio in anuncios:
        card = {}

        card['value'] = anuncio.find('p', class_='simple-card__price').get_text().replace('\n','').replace(' ','')

        infos = anuncio.find('ul', class_='feature__container simple-card__amenities').findAll('li')
        for info in infos:
            card[info.get('class')[2].split('-')[-1]] = info.get_text().split()[0]
            
        

        cards.append(card)


df = pd.DataFrame(cards)        


In [10]:

df.head(60)

,value,areas,bedrooms,spaces,bathrooms
0,R$134.990,32,1,0,1
1,R$222.000,46,2,1,1
2,R$975.000,204,4,2,4
3,R$1.400.000,380,4,4,5
4,R$1.300.000,390,5,4,5
5,R$1.200.000,1000,5,3,5
6,R$388.800,120,3,3,3
7,R$215.000,200,4,3,5
8,R$750.000,420,6,4,5
9,R$300.000,170,3,2,3


## Pequenos tratamentos antes de exportar

In [11]:
df = df.drop_duplicates()
df['value'] = df['value'].str.strip('R$')
df['value']

0         134.990
1         222.000
2         975.000
3       1.400.000
4       1.300.000
          ...    
1675    4.500.000
1676      950.000
1677    6.800.000
1678      900.000
1679    5.000.000
Name: value, Length: 1587, dtype: object

In [12]:
df['value'] = df['value'].str.replace('.','')

C:\Users\flpss\AppData\Local\Temp/ipykernel_11768/1939527559.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['value'] = df['value'].str.replace('.','')


In [13]:
df[['value', 'areas','bedrooms','spaces','bathrooms']] = df[['value', 'areas','bedrooms','spaces','bathrooms']].astype(float)

In [18]:
df.reset_index(drop=True, inplace=True)
df

,value,areas,bedrooms,spaces,bathrooms
0,134990.0,32.0,1.0,0.0,1.0
1,222000.0,46.0,2.0,1.0,1.0
2,975000.0,204.0,4.0,2.0,4.0
3,1400000.0,380.0,4.0,4.0,5.0
4,1300000.0,390.0,5.0,4.0,5.0
...,...,...,...,...,...
1582,4500000.0,600.0,4.0,6.0,6.0
1583,950000.0,306.0,3.0,3.0,4.0
1584,6800000.0,580.0,6.0,6.0,7.0
1585,900000.0,409.0,8.0,2.0,5.0


In [20]:
df.to_csv('C:\Users\flpss\Documents\Casas_Guaruja.csv', sep=';', index=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_11768/2915764794.py, line 1)